# **SQL Modernization What The Hack**

## **Challenge 2 - Intelligent Query Processing**

## Introduction

Wide World Importers has been motivated by the migration to the cloud to better understand database performance and requires your team's expertise. They are hoping to understand how to better evaluate problem spots, and take advantage of features within SQL Server 2019+ that might help.  Additionally, as part of their modernization effort, they feel this an ideal time to leverage Notebooks.

## Step 0 - Configuration

To isolate the difference in performance the team is seeing, all of these tests should be performed on the team's "on-premises" server -- we will switch database compatability appropriately to illustrate how the query performs on SQL 2016 vs SQL 2019, and the actual execution time is easier to compare as the hardware is identical.  Further, it will be easier to visualize the performance changes by using Query Store.  This may also be tested between on premises and the migrated server, but there may be other performance characteristics that impact performance.

## Step 1 - Create the test stored procedure

To demonstrate one of the performance anomalies WWI's developers are seeing, they've created the following code snippet that is similar to what is used in some of their analytical queries. 

This procedure uses a table variable populated from a user table and then joins it with a user table to provide output. T-SQL functions like COUNT and SUM are often seen in analytic queries. Note: In this example the TOP 1 T-SQL syntax is used so that the procedure only produces 1 row. This is only done to make the output easier to read in this workshop since this procedure will be executed multiple times. Normal execution of this procedure may not include TOP.

There is also a query to bring all pages from Sales.Invoices into buffer pool cache to ensure the comparisons are fair.

In [1]:
USE WideWorldImporters
GO
CREATE or ALTER PROCEDURE [Sales].[CustomerProfits]
AS
BEGIN
-- Declare the table variable
DECLARE @ilines TABLE
(	[InvoiceLineID] [int] NOT NULL primary key,
	[InvoiceID] [int] NOT NULL,
	[StockItemID] [int] NOT NULL,
	[Description] [nvarchar](100) NOT NULL,
	[PackageTypeID] [int] NOT NULL,
	[Quantity] [int] NOT NULL,
	[UnitPrice] [decimal](18, 2) NULL,
	[TaxRate] [decimal](18, 3) NOT NULL,
	[TaxAmount] [decimal](18, 2) NOT NULL,
	[LineProfit] [decimal](18, 2) NOT NULL,
	[ExtendedPrice] [decimal](18, 2) NOT NULL,
	[LastEditedBy] [int] NOT NULL,
	[LastEditedWhen] [datetime2](7) NOT NULL
)

-- Insert all the rows from InvoiceLines into the table variable
INSERT INTO @ilines SELECT * FROM Sales.InvoiceLines

-- Find my total profile by customer
SELECT TOP 1 COUNT(i.CustomerID) as customer_count, SUM(il.LineProfit) as total_profit
FROM Sales.Invoices i
INNER JOIN @ilines il
ON i.InvoiceID = il.InvoiceID
GROUP By i.CustomerID
END
GO
-- Pull these pages into cache to make the comparison fair based on a warm buffer pool cache
SELECT COUNT(*) FROM Sales.Invoices

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.057

(No column name)
70510


## Step 2 - Run the stored procedure with database compatibility of 130

To begin, the script will ensure the database is in a compatibility mode of 130 (SQL 2016).  The script also turns off rowcount messages to be returned to the client to reduce network traffic for this test. Then the script executes the stored procedure. Notice the syntax of **GO 25**. This is a client tool tip that says to run the batch 25 times (avoids having to construct a loop).

When you click Play to run the script look for these messages on the total elapsted time (your time may vary)

```
Beginning execution loop
Batch execution completed 25 times...
Total execution time: 00:00:40.3520665
```

In [2]:
USE master
GO
ALTER DATABASE wideworldimporters SET compatibility_level = 130
GO
USE WideWorldImporters
GO
SET NOCOUNT ON
GO
EXEC [Sales].[CustomerProfits]
GO 25
SET NOCOUNT OFF
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Beginning execution loop

...

customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


## Step 3 - Run the stored procedure with database compatibility of 150

Now let's run the same exact test but with database compatibility of 150. You will not make any changes to the stored procedure.

The script should execute far faster than before. 

When you click Play to run the script look for these messages on the total elapsted time (your time may vary)

```
Beginning execution loop
Batch execution completed 25 times...
Total execution time: 00:00:10.9975239
```

In [3]:
USE master
GO
ALTER DATABASE wideworldimporters SET compatibility_level = 150
GO
USE WideWorldImporters
GO
SET NOCOUNT ON
GO
EXEC [Sales].[CustomerProfits]
GO 25
SET NOCOUNT OFF
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Beginning execution loop

...

customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


customer_count,total_profit
406,147225.15


## Evaluation

Compare the total execution time with a compat level of 130 vs 150.  Now that we've (hopefully) seen the performance is substantially better on SQL 2019, it's time to analyze why so WWI can learn how to optimize their queries.  To complete this exercise, use Query Store to evaluate the different queries.  What is different about the execution plans?  What accounts for the difference?